<h1 style="text-align: center;">SEMINARIO Nº2:</h1>
<h1 style="text-align: center;">Estabilidad química de fármacos</h1>

Se conoce por **estabilidad** a la capacidad de un IFA o medicamento de mantener sus propiedades originales, por un tiempo determinado y dentro de especificaciones de calidad requeridas. Si bien existen diversos tipos de estabilidad (ej.: física, biológica, toxicológica, biofarmacéutica, etc.), nos centraremos en analizar la ***estabilidad química*** de fármacos.

# **Reacciones de degradación de fármacos**

In [1]:
from functions_Sem2 import *

Utilizaremos las siguientes funciones para identificar si los compuestos de interés poseen o no grupos hidrolizables u oxidables; y posteriormente podremos visualizar la estructura 2D y 3D de los productos de degradación.

La función *hydrolysis_products(input_smiles)* necesita como input el SMILES de la molécula a analizar, y genera como resultados la estructura 3D tanto del reactivo como de sus productos.

Podemos evaluar los compuestos uno a uno, tal como se ejemplifica abajo:

In [ ]:
input_smiles = "O=C(OCCN(CC)CC)C1=CC=C(N)C=C1"
hydrolysis_products(input_smiles)

O analizar el set completo, aplicando lo que se conoce como '*loop for*':

In [5]:
data_mols = {'mols': ['O=C(OCCN(CC)CC)C1=CC=C(N)C=C1', 'O=C(NCCN(CC)CC)C1=CC=C(N)C=C1',
'O=C(NCCN(CC)CC)C1=CC(Cl)=C(N)C=C1OC',
'COC1=CC=C(N(C(C2=CC=C(Cl)C=C2)=O)C(C)=C3CC(O)=O)C3=C1',
'OC1=C(O)C=C(C[C@@H](C(O)=O)N)C=C1',
'NC1=CC=C(S(NC(C)=O)(=O)=O)C=C1',
'CC(C)C1=C(C(=C(N1CCC(CC(CC(=O)O)O)O)C2=CC=C(C=C2)F)C3=CC=CC=C3)C(=O)NC4=CC=CC=C4',
'O=C1C(O)=C(O)C(C(CO)O)O1']}
df = pd.DataFrame(data_mols)

In [ ]:
for input_smiles in df['mols']:
  hydrolysis_products(input_smiles)

De manera análoga, podemos analizar ahora los fármacos susceptibles a sufrir oxidación, empleando la función oxidize_molecule()

In [ ]:
input_smiles = "OC1=C(O)C=C(C[C@@H](C(O)=O)N)C=C1"
oxidize_molecule(input_smiles)

In [ ]:
for input_smiles in df['mols']:
  oxidize_molecule(input_smiles)

# **Cinética Química**

Además de identificar grupos funcionales susceptibles a la degradación química, es de relevancia farmacéutica conocer la velocidad con la que se producen las reacciones de degradación; principalmente para definir períodos en los que fármacos/medicamentos puedan usarse de manera segura. Para ello, es necesario trabajar con conceptos asociados a la **cinética química**.

Como sabemos, para determinar el orden de una reacción necesitamos información de cómo varía la concentración en función del tiempo. Utilizando la función *analizar_cineticas()* podremos obtener los resultados directamente, tan solo generando la tabla o el *dataframe* correspondiente.

In [ ]:
# Ejemplo de uso con concentración
data_concentracion = {'tiempo': [0, 3, 6, 10, 30, 50, 70, 90],
                      'concentracion': [230, 228.1359, 227.2883, 226.3333, 220.6868, 207.3255, 198.4291, 190.2132]}
df_concentracion = pd.DataFrame(data_concentracion)

analizar_cineticas(df_concentracion)

Como pueden ver, los valores de r2 son muy cercanos entre sí, y el resultado entonces carece de precisión. Por eso, en la práctica, los ensayos siempre se replican -al menos- por duplicado o triplicado.

In [ ]:
data_concentracion = {'tiempo': [0, 3, 6, 10, 30, 50, 70, 90],
                      'concentracion1': [235, 229.1359, 227.2883, 226.3333, 220.6868, 207.3255, 198.4291, 190.2132],
                      'concentracion2': [230, 226.4, 227.8, 225.2, 217.4, 207.6, 198.7, 190.8],
                      'concentracion3': [233, 228.6, 221.0, 226.7, 211.0, 205.9, 199.5, 186.7]}
df_concentracion = pd.DataFrame(data_concentracion)

# Calcular la columna concentracion_promedio
df_concentracion['concentracion_promedio'] = df_concentracion[['concentracion1', 'concentracion2', 'concentracion3']].mean(axis=1)

# Calcula y asigna la desviación estándar para cada columna
df_concentracion['std_concentracion'] = df_concentracion[['concentracion1', 'concentracion2', 'concentracion3']].std(axis=1)
df_concentracion['std_log_concentracion'] = np.log(df_concentracion[['concentracion1', 'concentracion2', 'concentracion3']]).std(axis=1)
df_concentracion['std_inversa_concentracion'] = (1 / df_concentracion[['concentracion1', 'concentracion2', 'concentracion3']]).std(axis=1)

analizar_cineticas_multiple(df_concentracion)



Sin embargo, en la práctica se suelen emplear diversas metodologías para determinar esas concentraciones. Una de las más económicas y versátiles es la absorbancia UV-Visible. Gracias a la Ley de Lambert & Beer sabemos que - bajo ciertas condiciones - absorbancia es es directamente proporcional a concentración.

En este contexto, la función *analizar_cineticas()* también nos brinda la posibilidad de determinar órdenes de reacción a partir de Abs, debiendo especificar para ello los valores del coeficiente de absortividad molar o de extinción y el paso de celda.

In [ ]:
# Ejemplo de uso con absorbancia
data_absorbancia = {'tiempo': [0, 20, 40, 70, 90, 120, 150, 180],
                    'absorbancia': [0.83, 0.71, 0.60, 0.55, 0.47, 0.41, 0.32, 0.24]}
df_absorbancia = pd.DataFrame(data_absorbancia)

# Puedes proporcionar el coeficiente de extinción y la longitud de camino como argumentos adicionales
coef_extincion = 3400
paso_celda = 1.20

analizar_cineticas(df_absorbancia, coef_extincion, paso_celda)

Además de determinar órdenes de reacción y constantes de velocidad, hay otros parámetros cinéticos que son de relevancia en Farmacia. Entre ellos destacan los tiempos de vida media y tiempos de vida útil.

Las funciones *calcular_tiempo_vida_util()* y *calcular_tiempo_vida_media()* nos permiten obtener esos valores. Considerando que según el orden de reacción la fórmula cambia, es necesario proveerles 4 inputs:
- Orden de reacción: 0, 1 o 2.
- Constante de velocidad
- Concentración inicial (cuando corresponda)
- Tiempo de vida media o útil.


A modo de ejemplo, consideremos que sabemos que una reacción de hidrólisis sigue un orden 1, con una k = 1.5e-05 s-1 a 25ºC. Si queremos conocer el tiempo de vida útil aplicando la función deberíamos aplicar:

In [ ]:
k_calc, c0_calc, t90_calc  = calcular_tiempo_vida_util(1,1.5e-05,'','no')
print(f"Tiempo de vida útil t(90): {round(t90_calc,2)}")

En este caso, dejamos vacía ('') concentración inicial ya que para el cálculo de este tiempo para una reacción de Orden 1 no es necesario. E indicamos 'no' en el correspondiente a tiempo porque es el dato que NO tenemos.

¿Cómo la aplicaríamos si el orden de reacción fuese 0 con una concentración inicial de 0.4M?


In [ ]:
k_calc, c0_calc, t90_calc  = calcular_tiempo_vida_util(0,1.5e-05,0.4,'no')
print(f"Tiempo de vida útil t(90): {round(t90_calc,2)}")

Y si conociéramos que una reacción X sigue un orden 1 y a 25ºC tiene un t1/2 de 10000min, esta función nos permite conocer la constante de velocidad a esa temperatura según:

In [ ]:
k_calc, c0_calc, t90_calc  = calcular_tiempo_vida_util(1,'no','',10000)
print(k_calc)

Así pues, esta misma función nos permite calcular tiempos de vida media o util, concentraciones iniciales y/o constantes de velocidad a una temperatura determinada según los datos con los que contemos.

# **Condiciones que afectan estabilidad: Temperatura**

Gracias a la ecuación de Arrhenius podemos determinar numerosos parámetros cinéticos de relevancia trabajando a distintas temperaturas. Entre ellos, constantes de velocidad (k), energías de activación (Ea) y factor de colisiones (A). Dada su versatilidad e importancia se han definido distintas funciones que permitan obtener información variada según las necesidades y los datos con los que contamos.

Una posibilidad es contar con datos de constantes de velocidad a distintas temperaturas, y querer conocer los valores de Ea y/o A.

Hay que tener en cuenta que la ecuación de Arrhenius necesita que las T estén expresadas en grados Kelvin. Para simplicar el cálculo, la función directamente convierte los datos, siempre y cuando se especifique la unidad de temperatura, tal como se ejemplifica abajo.

In [ ]:
# Ejemplo de uso
arrhenius_data = {'k': [19, 14, 11, 5, 1.4, 0.95],
                  'Temperature': [100, 90, 80, 70, 60, 50]}

arrhenius_data_C = pd.DataFrame(arrhenius_data)

arrhenius_data_df_C = preparar_dataframe(arrhenius_data_C, temp_unit='C')

graficar_arrhenius(arrhenius_data_df_C)

De manera análoga, si ya contamos con los valores en grados Kelvin, deberíamos aplicar la función especificando K en vez de C, de la siguiente manera:

In [ ]:
arrhenius_data = {'k': [19, 14, 11, 5, 1.4, 0.95],
                  'Temperature': [373.15, 363.15, 353.15, 343.15, 333.15, 323.15]}

arrhenius_data_K = pd.DataFrame(arrhenius_data)

arrhenius_data_df_K = preparar_dataframe(arrhenius_data_K, temp_unit='K')

graficar_arrhenius(arrhenius_data_df_K)

Otro uso ampliamente asignado a la ecuación de Arrhenius es el Método de Degradación Acelerada, que nos permite identificar rápidamente parámetros cinéticos de interés tales como constantes de velocidad o Energía de activación trabajando a altas temperaturas.

Los datos necesarios para aplicar este método son:
- Dos valores de constantes de velocidad (k1 y k2) a dos temperaturas diferentes (T1 y T2).
- Energía de activación.

Cuando alguno de esos valores sea desconocido, se puede usar la ecuación para calcularlo. Por lo tanto, hemos desarrollado dos funciones para ello, denominadas *calcular_Ea()* y *calcular_k2()*

Entonces, si conocemos los pares k1-T1 y k2-T2, podremos calcular la Ea de la reacción según:

In [ ]:
#Ejemplo de uso
T1 = 273.15 + 80 #Tiene que estar en K!
T2 = 273.15 + 45 #Tiene que estar en K!
k1 = 0.0000766
k2 = 0.00000941

Ea_calculada = calcular_Ea(T1, T2, k1, k2)

print(f'Ea calculada: {Ea_calculada:.4f} cal/mol')

Y si ahora queremos conocer la constante de velocidad a otra temperatura (ej.: 25ºC), podemos aplicar:

In [ ]:
T3 = 273.15 + 25 #Tiene que estar en K!

k3_calculada = calcular_k(Ea_calculada, T1, T3, k1)
print(f'k3 calculada: {k3_calculada}')

Podríamos corroborar el resultado usando el otro par k-T.

In [ ]:
T3 = 273.15 + 25 #Tiene que estar en K!

k3_calculada = calcular_k(Ea_calculada, T2, T3, k2)
print(f'k3 calculada: {k3_calculada}')